# Step_6_Visualizing_Word2Vec and Sentiment Analysis
### Roy Madpis (319091526) And Michael Kobaivanov (206814485)

In this notebook we use all the data we collected so far to collect insights regarding the Brexit issue.
    
### 1. Sentiment Analysis



In [1]:
#! pip3 freeze > roy_libs1.txt

In [2]:
#! pip install --trusted-host=pypi.org --trusted-host=files.pythonhosted.org --user gensim
#! pip install dask
#! pip install tabulate
#! pip install nltk
#! pip install langid # for language detection
# ! pip install contractions #for cleaning the text - converting abbriviations to long text for example

### downloading the 4.5M tweets with sentiment
#! curl --remote-name-all https://www.clarin.si/repository/xmlui/bitstream/handle/11356/1135{/Brexit_tweets_stance.zip}


ERROR: Invalid requirement: '#'


In [ ]:
import pandas as pd, numpy as np, os
import seaborn as sns

import os.path
import matplotlib.pyplot as plt
import string
import psutil
import sys, requests
import time, datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from dask import dataframe as dd
import collections
from collections import Counter
from tqdm.notebook import tqdm_notebook

#for readin zip files
import zipfile
import bz2
import itertools
import codecs
import io

import pickle
from scipy import spatial
from sklearn.decomposition import LatentDirichletAllocation
import gensim
import inspect
import contractions
import langid #for language detection
#import logging
#logging.basicConfig(format ='%(asctime)s: %(levelname)s: %(message)s', level = logging.ERROR)

#stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


#####################################
data_folder = "data_folder"

In [ ]:
### reading the stopwords file / downloading it and add some more stopwords

stopwords_file_name =  os.path.join(data_folder, "StopWords")
stopwords_url = "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords"


stop_words_to_add = ["http", "https", "rt", "co", "vrkhaxde", "oeblog", "rln", "simonjhix", "cctvqfr",
"dcpj", "xvy", "mycekvwlxr", "imydbvvwji", "kkd", "rwp","yfc","fus","tmawgoafhb","edzmidvpel", "brexit.", "brexit,",
"xmljwysih", "lnfgc", "https//t.co/gx1lpjz5ux", "gvabanwvhi", "fmgynqxrs", "https//t.co/zpwcyltwgk", "https//t.co/eacccme91i",
"https//t.co/memxwp57du", "https//t.co/kgjzyab6vu", "vxpgogryp", "https//t.co/l7cfc70l1w"] ### add more stopwords you want
#################################################################
if not os.path.isfile(stopwords_file_name):
    StopWords = requests.get(stopwords_url).text.split()
    with open(stopwords_file_name,'w+t', encoding='utf-8') as out_file:
        out_file.write(' '.join(StopWords))
else:
    with open(stopwords_file_name,'rt', encoding='utf-8') as in_file:
        StopWords = in_file.readline().split()
StopWords = set(StopWords)

for word_i in stop_words_to_add:
    StopWords.add(word_i)

## 5.1.a Reading Key Opinion leaders tweets:


a. We split the ~1M tweets data (data_folder/tweets_table_all_with_sentimet.csv) by the year the tweet was published, train a Word2Vec model for each year alone (thus have word2vec model that was trained on tweets from 2014 / model that was trained on tweets from 2015, etc) we also have one Word2Vec model that was trained on all the data.

In [ ]:
dir_with_all_tweets = os.path.join(data_folder,"Predicted_sentiment_tables")

print("Step 1: Reading all the csv files")
tweets_tables = []
csv_files_evaluated = []
for root,dirs,files in os.walk(dir_with_all_tweets):
    for file in files:
        if file.endswith(".csv"): #if the file is csv
            csv_files_evaluated.append(file)
            file_location = os.path.join(dir_with_all_tweets, file)
            tweets_tables.append(pd.read_csv(file_location))

tweets_table_all_with_sentimet_0 = pd.concat(tweets_tables)
print("There are:", tweets_table_all_with_sentimet_0.shape[0], "Tweets")

tweets_table_all_with_sentimet_0 = tweets_table_all_with_sentimet_0.drop(labels = ["Unnamed: 0", "index_num"], axis = 1)
tweets_table_all_with_sentimet_0.head()

### Reading the table with 300k tweets with sentiments
This is the data that we read from twitter using th tweet-ids) [this data is out of the corpus of 4.5M tweet-ids with sentiment]

We need to preprocess that table so it would be in the same format and shape as the table we have above

In [ ]:
data_folder_name_for_sentiment = "data_folder/tweets_4_5M/tweet_ids_with_sentiment"

print("Step 1: Reading all the csv files")
tweets_tables = []
csv_files_evaluated = []
for root,dirs,files in os.walk(data_folder_name_for_sentiment):
    for file in files:
        if file.endswith(".csv"): #if the file is csv
            csv_files_evaluated.append(file)
            file_location = os.path.join(data_folder_name_for_sentiment, file)
            tweets_tables.append(pd.read_csv(file_location))

tweets_with_sentiment_300k = pd.concat(tweets_tables)
print("There are:", tweets_with_sentiment_300k.shape[0], "Tweets")

### preprocess

## change the sentiment column name
tweets_with_sentiment_300k = tweets_with_sentiment_300k.rename({"sentimnt":"Predicted_sentiment"}, axis = 1)

tweets_with_sentiment_300k["text_tokens"] = 0

def take_only_10_first_char(x):
    return(x[0:10])
tweets_with_sentiment_300k["created_at_date"] = tweets_with_sentiment_300k["created_at"].apply(take_only_10_first_char)
tweets_with_sentiment_300k["created_at_date"] = pd.to_datetime(tweets_with_sentiment_300k["created_at_date"])

tweets_with_sentiment_300k["Year_tweet"] = pd.DatetimeIndex(tweets_with_sentiment_300k['created_at']).year
tweets_with_sentiment_300k = tweets_with_sentiment_300k.drop(labels = ["Unnamed: 0","tweet_id"], axis = 1)

tweets_with_sentiment_300k.head()

In [ ]:
print(tweets_table_all_with_sentimet_0.shape)
print(tweets_with_sentiment_300k.shape)

In [ ]:
### combining the two tables into one big table
tweets_table_all_with_sentimet = pd.concat([tweets_table_all_with_sentimet_0, tweets_with_sentiment_300k])
print("Total number of tweets with sentiment =", tweets_table_all_with_sentimet.shape[0])

### remove duplicates
tweets_table_all_with_sentimet = tweets_table_all_with_sentimet.drop_duplicates(subset=['author_id_new', 'conv_id_new', "id_new", "created_at", "public_metrics.like_count", "text"])
print("Total number of tweets with sentiment not duplicate =", tweets_table_all_with_sentimet.shape[0])

tweets_table_all_with_sentimet.head()

In [158]:
# num_of_rows = tweets_table_all_with_sentimet.shape[0]
# num_groups = 100
# print("Number or rows in each group:", round(num_of_rows/num_groups,1))
# split_groups = np.array_split(range(num_of_rows), num_groups)

Number or rows in each group: 9.8


### Preprocess the filtered tweets
After we read the tweets filtered table, we need to perform several cleaning / preprocessing on those filtered tweets

In [ ]:
# def clean_text(x):
#     from gensim.utils import simple_preprocess
#     import contractions
#     x = contractions.fix(x)
#     x = ' '.join(simple_preprocess(x))
#     return x

### Pre-processing columns conv_id_new and author_id_new
tweets_table_all_with_sentimet['conversation_id'] = tweets_table_all_with_sentimet['conv_id_new'].apply(lambda x: x.replace("conv_id: ", ""))
tweets_table_all_with_sentimet['author_id'] = tweets_table_all_with_sentimet['author_id_new'].apply(lambda x: x.replace("author_id: ", ""))
#tweets_table_all_with_sentimet['clean_text'] = tweets_table_all_with_sentimet['text'].apply(clean_text)
tweets_table_all_with_sentimet = tweets_table_all_with_sentimet[tweets_table_all_with_sentimet['text'].notna()] #remove rows with nan values in the text column

tweets_table_all_with_sentimet['clean_text'] = tweets_table_all_with_sentimet['text']

if "Year_tweet" in tweets_table_all_with_sentimet.columns:
    years_tweets = tweets_table_all_with_sentimet.Year_tweet
else:
    years_tweets = pd.DatetimeIndex(tweets_table_all_with_sentimet['created_at_date']).year
    tweets_table_all_with_sentimet["Year_tweet"] = years_tweets
### detecting the language of the text + preprocess it
language_classifications = []
tokens_roy = []
years_tweets = []
for row in tqdm_notebook(tweets_table_all_with_sentimet.clean_text): #tqdm_notebook - to pring progress bar
    #language_classification, score = langid.classify(row)
    #language_classifications.append(language_classification)

    data_clean = row.lower().replace("#", "").replace("@", "").replace("?", "").replace("\"", "").replace("\'", "").replace(":", "")
    data_clean = data_clean.split()
    data_clean = [token for token in data_clean if token not in StopWords]
    data_clean = [token for token in data_clean if len(token) >=2]
    tokens_roy.append(data_clean) #

texts = tweets_table_all_with_sentimet.text
text_tokens = tweets_table_all_with_sentimet.text_tokens

In [7]:
# if "language" in filtered_table.columns:
#     language_classifications.append(language_classification)

In [ ]:
print(len(language_classifications))
print(len(tokens_roy))

### splitting the tweets to the relevant years

In [10]:
#years_tweets = pd.DatetimeIndex(tweets_table_all_with_sentimet['created_at_date']).year
years_tweets = tweets_table_all_with_sentimet['Year_tweet']

In [11]:
years_tweets_2014 = [years_tweets==2014]
years_tweets_2014 = np.asarray(years_tweets_2014).reshape(-1,1)

years_tweets_2015 = [years_tweets==2015]
years_tweets_2015 = np.asarray(years_tweets_2015).reshape(-1,1)

years_tweets_2016 = [years_tweets==2016]
years_tweets_2016 = np.asarray(years_tweets_2016).reshape(-1,1)

years_tweets_2017 = [years_tweets==2017]
years_tweets_2017 = np.asarray(years_tweets_2017).reshape(-1,1)

years_tweets_2018 = [years_tweets==2018]
years_tweets_2018 = np.asarray(years_tweets_2018).reshape(-1,1)

years_tweets_2019 = [years_tweets==2019]
years_tweets_2019 = np.asarray(years_tweets_2019).reshape(-1,1)

years_tweets_2020 = [years_tweets==2020]
years_tweets_2020 = np.asarray(years_tweets_2020).reshape(-1,1)

years_tweets_2021 = [years_tweets==2021]
years_tweets_2021 = np.asarray(years_tweets_2021).reshape(-1,1)

In [10]:
tokens_df = pd.DataFrame(tokens_roy)#[years_tweets_2018]
tokens_df_2014 = tokens_df[years_tweets_2014]
tokens_df_2015 = tokens_df[years_tweets_2015]
tokens_df_2016 = tokens_df[years_tweets_2016]
tokens_df_2017 = tokens_df[years_tweets_2017]
tokens_df_2018 = tokens_df[years_tweets_2018]
tokens_df_2019 = tokens_df[years_tweets_2019]
tokens_df_2020 = tokens_df[years_tweets_2020]
tokens_df_2021 = tokens_df[years_tweets_2021]

tokens_df_all = tokens_df

In [12]:
print("2014:", tokens_df_2014.shape)
print("2015:", tokens_df_2015.shape)
print("2016:", tokens_df_2016.shape)
print("2017:", tokens_df_2017.shape)
print("2018:", tokens_df_2018.shape)
print("2019:", tokens_df_2019.shape)
print("2020:", tokens_df_2020.shape)
print("2021:", tokens_df_2020.shape)
print("All:", tokens_df_all.shape)

NameError: name 'tokens_df_2014' is not defined

In [ ]:
### this function takes the data frame and takes all the None values out + transform it to list of list (each row becomes a list)
def tokens_df_new(tokens_df):
    tokens_df_new = []
    for list_i in tokens_df.values.tolist():
        list_i_new = []
        for i in list_i:
            if i != None:
                list_i_new.append(i)

        tokens_df_new.append(list_i_new)
    return(tokens_df_new)

In [ ]:
tokens_df_new_2014 = tokens_df_new(tokens_df_2014)
tokens_df_new_2015 = tokens_df_new(tokens_df_2015)
tokens_df_new_2016 = tokens_df_new(tokens_df_2016)
tokens_df_new_2017 = tokens_df_new(tokens_df_2017)
tokens_df_new_2018 = tokens_df_new(tokens_df_2018)
tokens_df_new_2019 = tokens_df_new(tokens_df_2019)
tokens_df_new_2020 = tokens_df_new(tokens_df_2020)
tokens_df_new_2021 = tokens_df_new(tokens_df_2021)
tokens_df_all = tokens_df_new(tokens_df_all)

In [14]:
len(tokens_df_new_2014)

NameError: name 'tokens_df_new_2014' is not defined

In [13]:
import warnings
warnings.filterwarnings('ignore')

import warnings
warnings.filterwarnings('default')


## EDA

In [ ]:
# word cloud


In [18]:
tweets_table_all_with_sentimet.clean_text

0        rt downingstreet more than countries places in...
1        rt gavinwilliamson want to thank all those ser...
2        rt ukgovwales welsh secretary aluncairns exten...
3                          tickettattle tfwrail sure thing
4        wpl_official barrytownunited the_nomads balato...
                               ...                        
62542    RT @RichardWellings: Good the Brexit debate is...
62543    RT @BelTel: EU Referendum: Northern Ireland sa...
62544    RT @K8teMorgan: Regardless of your voting pref...
62545    RT @Vote_LeaveMedia: PM's former strategy chie...
62546    RT @Keroppo: One of the main donors of the Rem...
Name: clean_text, Length: 1285805, dtype: object

In [17]:
#wordcloud install from anaconda prompt:
#conda install -c conda-forge wordcloud 

from wordcloud import WordCloud

In [19]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os


tweets_texts = list(map(lambda x: x.lower(), tweets_table_all_with_sentimet.clean_text)) #lower_case
#remove punctuation and save in one big string (so we can split all the corpus into words)
tweets_texts = str(tweets_texts).translate(str.maketrans('', '', string.punctuation))


In [ ]:
tweets_texts

In [ ]:
print("Wordcloud - Positive tweets")
wordcloud_positive = WordCloud(max_font_size=40)
wordcloud_positive = wordcloud_positive.generate(positive_tweets)

fig = plt.figure(figsize=(12,9))
plt.imshow(wordcloud_positive, interpolation="bilinear")
plt.axis("off")
plt.show();
fig.savefig('local_data/486.full.png', transparent=True, format='png')

## Sentiment analysis

Here we present the EDA we conducted using the sentiment prediction (that was the outcome of the sentiment model)